In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import logging

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("stanfordnlp/imdb")
train = dataset.get("train")
test = dataset.get("test")

In [3]:
num_labels = len(train.unique("label"))
name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(name)

In [4]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process,
    batched=True,
    num_proc=2,
)
test_dataset = test.map(
    pre_process,
    batched=True,
    num_proc=2,
)

In [5]:
def collate_fn(batch):
    inputs = {"input_ids": [], "attention_mask": []}
    labels = []
    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [6]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [7]:
lr = 5e-5
weight_decay = 1e-4

model = AutoModelForSequenceClassification.from_pretrained(name, num_labels=num_labels)
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

In [8]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    epochs=3,
    store_update_metrics=True,
    unpack_inputs=True,
    device=device,
    to_fn=to,
    path="../training/imdb-distil-bert-sgdm.h5",
    verbose=True,
    profile=True,
    logger_name="sgdm"
)

In [9]:
sgdm_metrics = trainer.train()

2024-12-18 19:52:20,922 - INFO - model using cuda
2024-12-18 19:52:24,325 - INFO - training started
2024-12-18 19:56:23,981 - INFO - (epoch: 1/3): (train loss: 0.6833, test loss: 0.6652, train acc: 59.72%, test acc: 79.36%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5989.0 MB)
(duration info): (epoch duration: 0:03:59, elapsed time: 0:03:59)
2024-12-18 20:00:21,471 - INFO - (epoch: 2/3): (train loss: 0.5660, test loss: 0.3770, train acc: 80.46%, test acc: 86.30%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5990.0 MB)
(duration info): (epoch duration: 0:03:57, elapsed time: 0:07:57)
2024-12-18 20:04:15,649 - INFO - (epoch: 3/3): (train loss: 0.3202, test loss: 0.2746, train acc: 87.52%, test acc: 88.82%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5990.0 MB)
(duration info): (epoch duration: 0:03:54, elapsed time: 0:11:51)
2024-12-18 20:04:15,650 - INFO - training complete
2024-12-18 20:04:15,651 - INFO - tra